### 1. 安装HuggingFace 并下载模型到本地

In [6]:
!pip install huggingface-hub -Uqq 
!pip install -U sagemaker

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [42]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_bc2_7b_model")


In [43]:
local_model_path.mkdir(exist_ok=True)
model_name = "baichuan-inc/Baichuan2-7B-Chat"
# commit_hash = "670d17ee403f45334f53121d72feff623cc37de1"

In [4]:
snapshot_download(repo_id=model_name, 
                  # revision=commit_hash,
                  cache_dir=local_model_path)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

'LLM_bc2_7b_model/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/229e4eb1fab7f6aef90a2344c07085b680487597'

### 2. 把模型拷贝到S3为后续部署做准备

In [44]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [45]:
s3_model_prefix = "LLM-RAG/workshop/LLM_bc2_7b_stream_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_bc2_7b_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_bc2_7b_stream_deploy_code
model_snapshot_path: LLM_bc2_7b_model/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/229e4eb1fab7f6aef90a2344c07085b680487597


In [7]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_bc2_7b_model/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/229e4eb1fab7f6aef90a2344c07085b680487597/.gitattributes to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/LLM_bc2_7b_stream_model/.gitattributes
upload: LLM_bc2_7b_model/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/229e4eb1fab7f6aef90a2344c07085b680487597/README.md to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/LLM_bc2_7b_stream_model/README.md
upload: LLM_bc2_7b_model/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/229e4eb1fab7f6aef90a2344c07085b680487597/Community License for Baichuan2 Model.pdf to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/LLM_bc2_7b_stream_model/Community License for Baichuan2 Model.pdf
upload: LLM_bc2_7b_model/models--baichuan-inc--Baichuan2-7B-Chat/snapshots/229e4eb1fab7f6aef90a2344c07085b680487597/quantizer.py to s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/LLM_bc2_7b_stream_model/quantizer.py
upload: LLM_bc2_7b_model/models--baichuan-inc--B

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [46]:
# inference_image_uri = (
#     f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [47]:
!mkdir -p LLM_bc2_7b_stream_deploy_code

In [48]:
%%writefile LLM_bc2_7b_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
STOP_flag = "[DONE]"


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    tokenizer = AutoTokenizer.from_pretrained(model_location, use_fast=False, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True).cuda() 
    model.generation_config = GenerationConfig.from_pretrained(model_location)

    return model, tokenizer


model = None
tokenizer = None
generator = None


def construct_message(history,prompt):
    message = []
    for question, answer in history:
        message.append({"role":"user","content":question})
        message.append({"role":"assistant","content":answer})
    message.append({"role":"user","content":prompt})
    return message

def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    model.generation_config.max_new_tokens = max_length
    model.generation_config.top_p = top_p
    
    messages = construct_message(history,prompt)
    res_generator = model.chat(tokenizer, messages,stream=True)
    for response in res_generator:
        this_response = response[size:]
        size = len(response)
        stream_buffer = { "outputs":this_response,"finished": False}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": [], "finished": True}
    


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    stream = data.get('stream')
    print(f'input prompt:{input_sentences}')    
    outputs = Output()
    if stream:
        outputs.add_property("content-type", "application/jsonlines")
        outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    else:
        messages = construct_message(history,input_sentences)
        response = model.chat(tokenizer, messages,stream=False)
        result = {"outputs": response, "history" : []}
        outputs.add_as_json(result)
        
    return outputs

Overwriting LLM_bc2_7b_stream_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [49]:
%%writefile LLM_bc2_7b_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/LLM_bc2_7b_stream_model/

Overwriting LLM_bc2_7b_stream_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [50]:
%%writefile LLM_bc2_7b_stream_deploy_code/requirements.txt
transformers==4.31.0
accelerate
colorama
bitsandbytes
sentencepiece
transformers_stream_generator
cpm_kernels

Overwriting LLM_bc2_7b_stream_deploy_code/requirements.txt


In [51]:
!rm model.tar.gz
!cd LLM_bc2_7b_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_bc2_7b_stream_deploy_code

LLM_bc2_7b_stream_deploy_code/
LLM_bc2_7b_stream_deploy_code/model.py
LLM_bc2_7b_stream_deploy_code/requirements.txt
LLM_bc2_7b_stream_deploy_code/serving.properties


In [52]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-631023274615/LLM-RAG/workshop/LLM_bc2_7b_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [53]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"bc2-7b-stream") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)


model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bc2-7b-stream-2023-09-15-06-03-38-304
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:631023274615:model/bc2-7b-stream-2023-09-15-06-03-38-304


In [54]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 15*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:631023274615:endpoint-config/bc2-7b-stream-2023-09-15-06-03-38-304-config',
 'ResponseMetadata': {'RequestId': '0a9e8851-2ff5-4794-8534-aa66117fcbfe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0a9e8851-2ff5-4794-8534-aa66117fcbfe',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '125',
   'date': 'Fri, 15 Sep 2023 06:03:43 GMT'},
  'RetryAttempts': 0}}

In [55]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:631023274615:endpoint/bc2-7b-stream-2023-09-15-06-03-38-304-endpoint


#### 持续检测模型部署进度

In [56]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:631023274615:endpoint/bc2-7b-stream-2023-09-15-06-03-38-304-endpoint
Status: InService


### 5. 模型测试

In [21]:
!pip install -U boto3 botocore

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/bf/b1/8c254dbb9e315f3b2f5a2a30667a08aed13fd9a00e07028eee701c7ce00d/boto3-1.28.47-py3-none-any.whl.metadata
  Obtaining dependency information for botocore from https://files.pythonhosted.org/packages/16/e7/5b9339e7d076ddacafe51614e1e48100a52ed3ef16465a6cd625e16bde86/botocore-1.31.47-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 1.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 37.2 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.42
    Uninstalling botocore-1.31.42:
      Successfully uninstalled botocore-1.31.42
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.42
    Uninstalling boto3-1.28.42:
      Successfully uninstalled boto3-1.28.42
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed

In [35]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 1024,
  "temperature": 0.5,
  "top_p":0.9
}

CPU times: user 4.86 ms, sys: 0 ns, total: 4.86 ms
Wall time: 4.87 ms


In [36]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

## Stream 

In [37]:
import time


# prompts1 = """what's the weather like there?"""
# history = [
#         [
#             "what is the capital city of china?",
#             "Beijing."
#         ]
#     ]
# prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
# 请问AWS Clean Rooms是多方都会收费吗？
# """
# prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
prompts1 = """床前明月光是谁写的"""
history = []
# endpoint_name = 'bc2-13b-stream-2023-09-07-10-59-15-557-endpoint'

start = time.time()
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : history,
                "stream":True
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            # print(resp)
            print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            # print(line)
            continue
print (f"time:{time.time()-start} s")

"床前明月光"这句诗出自唐代诗人杜甫的《静夜思》。time:4.209242343902588 s


## None Stream

In [38]:
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
# prompts1 = """床前明月光是谁写的"""
# prompts1 = """那里有什么景点"""
# history = [
#         [
#             "中国的首都是哪里",
#             "北京."
#         ]
#     ]
start = time.time()
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
            }
            ),
            ContentType="application/json",
        )

resp = response_model['Body'].read()
end = time.time()-start
print (f"\ntime:{end} s")
# print(resp.decode('utf8'))
print(json.loads(resp)['outputs'])


time:15.814193725585938 s
在遥远的未来，宇宙成为了人类主要的战场。星际战争不断爆发，各个星球的势力都在争夺更多的资源和领土。在这个时代，有一个名为“银河联邦”的庞大组织，他们拥有强大的科技力量，试图统一整个宇宙。然而，他们也面临着来自其他星球的威胁，如邪恶的“黑暗帝国”。

故事的主人公，杰克，是一名年轻的银河联邦士兵。他在一次任务中，意外地发现了黑暗帝国的秘密武器——一种可以改变时空的装置。这个发现让杰克意识到，黑暗帝国可能会利用这种武器来征服整个宇宙。为了阻止这一灾难，杰克决定潜入黑暗帝国的基地，摧毁这个装置。

在潜入过程中，杰克遇到了一位神秘的女子，她名叫莉莉。莉莉告诉杰克，她是黑暗帝国的一名间谍，她的任务是收集关于时空装置的信息。两人决定合作，共同阻止黑暗帝国的野心。

经过一系列惊心动魄的战斗和冒险，杰克和莉莉终于找到了时空装置。然而，他们发现，只有一个人能够使用这个装置，那就是黑暗帝国的皇帝。为了阻止黑暗帝国，杰克和莉莉决定绑架皇帝，迫使他放弃这个装置。

在一场激烈的战斗中，杰克和莉莉成功地绑架了皇帝。然而，他们并没有立即摧毁时空装置，而是选择了一个安全的地方，将皇帝关押起来。他们相信，只要黑暗帝国失去了皇帝，就无法继续扩张。

然而，银河联邦和其他星球的势力并没有放过杰克和莉莉。他们认为，这两个年轻人破坏了宇宙的和平，应该受到惩罚。于是，他们开始追捕杰克和莉莉，试图将他们绳之以法。

在这场追捕中，杰克和莉莉结识了许多志同道合的朋友，他们一起抵抗银河联邦和其他星球势力的压迫。在这个过程中，杰克和莉莉逐渐发现，原来黑暗帝国并不是唯一的敌人，银河联邦内部也存在许多问题。他们开始质疑自己的信仰和价值观，不知道到底谁才是真正的敌人。

最终，杰克和莉莉决定回到黑暗帝国的基地，摧毁时空装置。在他们离开之前，皇帝告诉他们一个惊人的秘密


In [38]:

def construct_message(history,prompt):
    message = []
    for question, answer in history:
        message.append({"role":"user","content":question})
        message.append({"role":"assistant","content":answer})
    message.append({"role":"user","content":prompt})
    return message

In [52]:
prompts1 = """那里有什么景点"""
history = [
        [
            "中国的首都是哪里",
            "北京."
        ]
    ]

In [53]:
messages=construct_message(history,prompts1)

In [54]:
def _parse_messages(messages, split_role="user"):
    system, rounds = "", []
    round = []
    for i, message in enumerate(messages):
        if message["role"] == "system":
            assert i == 0
            system = message["content"]
            continue
        if message["role"] == split_role and round:
            rounds.append(round)
            round = []
        round.append(message)
    if round:
        rounds.append(round)
    return system, rounds

In [81]:
system, rounds = _parse_messages(messages)
history_tokens = []
for round in rounds[::-1]:
    round_tokens = []
    
    for message in round:
        if message["role"] == "user":
            round_tokens.append(1)
        else:
            round_tokens.append(2)
        round_tokens.extend(message["content"])
    history_tokens = round_tokens + history_tokens  # concat left
print(history_tokens)

[1, '中', '国', '的', '首', '都', '是', '哪', '里', 2, '北', '京', '.', 1, '那', '里', '有', '什', '么', '景', '点']


In [73]:
round_tokens

[1, '中', '国', '的', '首', '都', '是', '哪', '里', 2, '北', '京', '.']

In [74]:
rounds[::-1]

[[{'role': 'user', 'content': '那里有什么景点'}],
 [{'role': 'user', 'content': '中国的首都是哪里'},
  {'role': 'assistant', 'content': '北京.'}]]

In [67]:
rounds

[[{'role': 'user', 'content': '中国的首都是哪里'},
  {'role': 'assistant', 'content': '北京.'}],
 [{'role': 'user', 'content': '那里有什么景点'}]]

#### 清除模型Endpoint和config

In [39]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [40]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [41]:
!aws sagemaker delete-model --model-name {model_name}